In [ ]:
import keras
from keras import layers
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

In [ ]:
from keras import models
from os.path import join

In [ ]:
batch_size = 24
embedding_dim = 50
image_size = 224
path_base = '../input/whale-categorization-playground/'
path_train = join(path_base,'train/train')
path_test = join(path_base,'test/test')
path_model = join(path_base,'resnet50_Model.hdf5')
path_csv = '../input/whale-categorization-playground/train.csv'

In [ ]:
class sample_gen(object):
    def __init__(self, file_class_mapping, other_class = "new_whale"): # file_class_mapping：key：图片名，value：Id
        self.file_class_mapping= file_class_mapping
        self.class_to_list_files = defaultdict(list) # <class 'list'>, {'w_a232f9e': ['33c25291.jpg', '67c96540.jpg'], 'w_1b6d171': ['64c99b7b.jpg'],
        self.list_other_class = [] # 其他类别图片名
        self.list_all_files = list(file_class_mapping.keys())  # 所有图片的文件名
        self.range_all_files = list(range(len(self.list_all_files)))  # 训练集就是0~6894的数字list

        for file, class_ in file_class_mapping.items():
            if class_ == other_class:
                self.list_other_class.append(file)
            else:
                self.class_to_list_files[class_].append(file)

        self.list_classes = list(set(self.file_class_mapping.values()))
        self.range_list_classes= range(len(self.list_classes))
        self.class_weight = np.array([len(self.class_to_list_files[class_]) for class_ in self.list_classes])
        self.class_weight = self.class_weight/np.sum(self.class_weight)

    def get_sample(self):
        class_idx = np.random.choice(self.range_list_classes, 1, p=self.class_weight)[0]
        examples_class_idx = np.random.choice(range(len(self.class_to_list_files[self.list_classes[class_idx]])), 2)
        positive_example_1, positive_example_2 = \
            self.class_to_list_files[self.list_classes[class_idx]][examples_class_idx[0]],\
            self.class_to_list_files[self.list_classes[class_idx]][examples_class_idx[1]]


        negative_example = None
        while negative_example is None or self.file_class_mapping[negative_example] == \
                self.file_class_mapping[positive_example_1]:
            negative_example_idx = np.random.choice(self.range_all_files, 1)[0]
            negative_example = self.list_all_files[negative_example_idx]
        return positive_example_1, negative_example, positive_example_2
    
def read_and_resize(filepath):
    im = Image.open((filepath)).convert('RGB')
    im = im.resize((image_size, image_size))
    return np.array(im, dtype="float32")


def augment(im_array):
    if np.random.uniform(0, 1) > 0.9:
        im_array = np.fliplr(im_array)
    return im_array

def gen(triplet_gen):
    while True:
        list_positive_examples_1 = []
        list_negative_examples = []
        list_positive_examples_2 = []

        for i in range(batch_size):
            positive_example_1, negative_example, positive_example_2 = triplet_gen.get_sample()
            path_pos1 = join(path_train, positive_example_1)
            path_neg = join(path_train, negative_example)
            path_pos2 = join(path_train, positive_example_2)
            
            positive_example_1_img = read_and_resize(path_pos1)
            negative_example_img = read_and_resize(path_neg)
            positive_example_2_img = read_and_resize(path_pos2)

            positive_example_1_img = augment(positive_example_1_img)
            negative_example_img = augment(negative_example_img)
            positive_example_2_img = augment(positive_example_2_img)
            
            list_positive_examples_1.append(positive_example_1_img)
            list_negative_examples.append(negative_example_img)
            list_positive_examples_2.append(positive_example_2_img)

        A = preprocess_input(np.array(list_positive_examples_1))
        B = preprocess_input(np.array(list_positive_examples_2))
        C = preprocess_input(np.array(list_negative_examples))
        
        label = None
        
        yield ({'anchor_input': A, 'positive_input': B, 'negative_input': C}, label)

In [ ]:
def triplet_loss(inputs, dist='sqeuclidean', margin='maxplus'):
    anchor, positive, negative = inputs
    positive_distance = K.square(anchor - positive)
    negative_distance = K.square(anchor - negative)
    if dist == 'euclidean':
        positive_distance = K.sqrt(K.sum(positive_distance, axis=-1, keepdims=True))
        negative_distance = K.sqrt(K.sum(negative_distance, axis=-1, keepdims=True))
    elif dist == 'sqeuclidean':
        positive_distance = K.sum(positive_distance, axis=-1, keepdims=True)
        negative_distance = K.sum(negative_distance, axis=-1, keepdims=True)
    loss = positive_distance - negative_distance
    if margin == 'maxplus':
        loss = K.maximum(0.0, 1 + loss)
    elif margin == 'softplus':
        loss = K.log(1 + K.exp(loss))
    return K.mean(loss)

def triplet_loss_np(inputs, dist='sqeuclidean', margin='maxplus'):
    anchor, positive, negative = inputs
    positive_distance = np.square(anchor - positive)
    negative_distance = np.square(anchor - negative)
    if dist == 'euclidean':
        positive_distance = np.sqrt(np.sum(positive_distance, axis=-1, keepdims=True))
        negative_distance = np.sqrt(np.sum(negative_distance, axis=-1, keepdims=True))
    elif dist == 'sqeuclidean':
        positive_distance = np.sum(positive_distance, axis=-1, keepdims=True)
        negative_distance = np.sum(negative_distance, axis=-1, keepdims=True)
    loss = positive_distance - negative_distance
    if margin == 'maxplus':
        loss = np.maximum(0.0, 1 + loss)
    elif margin == 'softplus':
        loss = np.log(1 + np.exp(loss))
    return np.mean(loss)

def check_loss():
    batch_size = 10
    shape = (batch_size, 4096)

    p1 = normalize(np.random.random(shape))
    n = normalize(np.random.random(shape))
    p2 = normalize(np.random.random(shape))
    
    input_tensor = [K.variable(p1), K.variable(n), K.variable(p2)]
    out1 = K.eval(triplet_loss(input_tensor))
    input_np = [p1, n, p2]
    out2 = triplet_loss_np(input_np)

    assert out1.shape == out2.shape
    print(np.linalg.norm(out1))
    print(np.linalg.norm(out2))
    print(np.linalg.norm(out1-out2))

In [ ]:
from sklearn.preprocessing import normalize
from keras import backend as K

In [ ]:
check_loss()

In [ ]:
from keras.applications import ResNet50   
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
# resnet_base= ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))   
resnet_base= ResNet50(weights=resnet_weights_path, include_top=False, input_shape=(224,224,3))

In [ ]:
resnet_base.summary()

In [ ]:
from keras.layers import Input, Dense, Dropout, Lambda, Convolution2D, MaxPooling2D, Flatten

In [ ]:
model = models.Sequential()
model.add(resnet_base)
model.add(Dropout(0.5))
model.add(layers.Dense(embedding_dim,activation='relu'))
model.add(layers.Lambda(lambda x: K.l2_normalize(x,axis=1)))

In [ ]:
model.summary()

In [ ]:
for layer in resnet_base.layers[150:]:
    layer.trainable = True
for layer in resnet_base.layers[:150]:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
from keras.models import Model

In [ ]:
input_shape = (image_size, image_size, 3)
anchor_input = Input(input_shape, name='anchor_input')
positive_input = Input(input_shape, name='positive_input')
negative_input = Input(input_shape, name='negative_input')
anchor_embedding = model(inputs=anchor_input)
positive_embedding = model(inputs=positive_input)
negative_embedding = model(inputs=negative_input)

inputs = [anchor_input, positive_input, negative_input]
outputs = [anchor_embedding, positive_embedding, negative_embedding]
       
triplet_model = Model(inputs, outputs)
triplet_model.add_loss(K.mean(triplet_loss(outputs)))


In [ ]:
from sklearn.model_selection import train_test_split
from collections import defaultdict
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

In [ ]:
data = pd.read_csv(path_csv)
train, test = train_test_split(data, train_size=0.7, random_state=1337)
file_id_mapping_train = {k: v for k, v in zip(train.Image.values, train.Id.values)}
file_id_mapping_test = {k: v for k, v in zip(test.Image.values, test.Id.values)}
gen_tr = gen(sample_gen(file_id_mapping_train))
gen_te = gen(sample_gen(file_id_mapping_test))

checkpoint = ModelCheckpoint(path_model, monitor='loss', verbose=1, save_best_only=True, mode='min') # 保存模型权重为 path_model
early = EarlyStopping(monitor="val_loss", mode="min", patience=10) # 当监测值不再改善时，该回调函数将中止训练
callbacks_list = [checkpoint, early]  # early

In [ ]:
from PIL import Image

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input

In [ ]:
def ShowImg(img):
    plt.figure()
    plt.imshow(img.astype('uint8'))
    plt.show()
    plt.close()
    
batch = next(gen_tr)

img = batch[0]['anchor_input'][0]
print(img.shape)
mean = [103.939, 116.779, 123.68]
img[..., 0] += mean[0]
img[..., 1] += mean[1]
img[..., 2] += mean[2]
img = img[..., ::-1]
ShowImg(img)

In [ ]:
a = np.array([ [[1,2,3] , [4,5,6] ],
              [[7,8,9] , [10,11,12] ] ])
print(a.shape)
a[..., 2] += 100
a = a[:,:,::-1]
print(a)

In [ ]:
from keras.layers import Input, Dense, Dropout, Lambda, Convolution2D, MaxPooling2D, Flatten
from keras.optimizers import Adam

In [ ]:
triplet_model.summary()

In [ ]:
triplet_model.compile(loss=None,
                      optimizer= keras.optimizers.Adam(lr = 0.0001),
                      metrics=['acc']
                     )
history = triplet_model.fit_generator(gen_tr, 
                              validation_data=gen_te, 
                              epochs=10, 
                              verbose=1, 
                              workers=1,
                              steps_per_epoch=100, 
                              validation_steps=20)

In [ ]:
plt.plot(history.history['loss'], label='traning_loss')
plt.plot(history.history['val_loss'], label='validation_loss', color = 'r')
plt.legend()
plt.show()